---
toc: true
comments: false
layout: post
title: Individual Review - Advik G
description: Recap of key events and learning
type: tangibles
courses: { compsci: {week: 10} }
---

# Issue Recap
I used [GitHub Issues](https://github.com/APCSP-RAGS/awsRAGS_flask/issues) to organize work for me and my partner, Srijan:
- [Starting Checklist](https://github.com/APCSP-RAGS/awsRAGS_flask/issues/2): This helped us stay on task and get organized while in the starting stages of development
- [Commit Summaries](https://github.com/APCSP-RAGS/awsRAGS_flask/issues/3): This summary was updated for each relevant commit. This is incredibly useful for charting our progress and looking at each commit easily. THe supplied text also served as more detailed commit messages. In general, this issue was used the most for organizing work done each day.

We didn't have many other issues as most of our task delegation was in communication through Slack and/or the checklist issue.

# Backend Deployment
I had a lot of struggles deploying our backend to AWS in the beginning, so I made a [blog post](https://advikg.github.io/student/2023/10/08/AWS-Struggles_IPYNB_2_.html) a while ago to explain what I faced, and hopefully help other people with the same issues.


## DNS
Registering a domain should've been a relatively easy process, but there were many struggles that people had, including me. First off, the AWS Route 53 layout looks complicated, to say the least. But when you find the correct spot by going to Hosted Zones > stu.nighthawkcodingsociety.com > Create Record, there's some problems in the directions. It tells you to create a CNAME record pointing to the domain stu.nighthawkcodingsociety.com, but what this does is redirect first to the site domain, then to the site IP. This chain redirection messes with some web interactions, so it's better to use the static IP address of the server: `3.130.255.192`
- Also, make sure **not** to edit anything else, it's best not to change default settings unless you know exactly what you're doing.

One more thing:
A **lot** of people have been using a third-party website called `Duck DNS` for their domains. While this isn't necessarily a bad thing, using Route 53 isn't that complicated, will allow classmates and Mr. Mortenson/Mr. Lopez to help troubleshoot your site, and helps them manage your sites easily. I'd highly suggest using Route 53 over Duck DNS.

## AWS/Docker Deployment
There's a few things that people needed to keep in mind when finally deploying their backend:
1. Make sure your port is **defined** in the docker files and **available** on the AWS instance(use `docker ps` to view ports currently in use)
2. Make sure that running `docker-compose up` on your host in the correct directory runs the webserver with no issue, on the right port
3. Once you are **finished** with the first few steps, you can clone the repo on the AWS instance and run the docker build
4. Make sure to tailor the NGINX configurations specifically towards your site, and don't forget to test the configs using `nginx -t`
5. After reloading the configs, make sure your domain is listed in the output for `certbot --nginx`
A problem I had was that my domain was not listed in the certbot output. I used ChatGPT for help and found that you can use certbot for a specific domain by running `sudo certbot --nginx -d ----.stu.nighthawkcodingsociety.com`(insert the right domain name there).
I found out that since my domain name had an underscore(_), certbot looked at it as invalid. I was able to quickly solve this by changing the domain in Route 53 and in the NGINX configurations, and reloading everything.

## General Tips
- Use sudo for all of your commands, this gets rid of any permission errors you might have
- Read the errors you have, don't just bug someone else to figure it out for you. ChatGPT and google are great tools for troubleshooting errors, and this way you can also learn more
- Make sure to reload nginx configurations or redo the docker build if you change any configurations
- Try not to mess with the instance(Putting random stuff in .bashrc, using a forkbomb, making annoying cronjobs), it makes it a lot less fun for everybody

Well, that's all I have. You can view our groups backend site [here](https://awsrags-flask.stu.nighthawkcodingsociety.com)

# Backend Development
Alright, now onto the actual development of the backend. First going into this, I was very confused. I slowly started to understand the structure of the server after experimenting with the Jokes API to make a `Create` method.

## Testing with Jokes
I first started with the `jokes.py` file in the `model` folder, where I defined a function that added a joke to the dictionary:

In [ ]:
# Function to add jokes(for create method)
def createJoke(joke):
    item_id = len(jokes_data)
    jokes_data.append({"id": item_id, "joke": joke, "haha": 0, "boohoo": 0})

Then, I added some code to use the JSON request body in the API endpoint:

In [ ]:
class JokesAPI:
    # Method to create joke, 
    class _Create(Resource):
        def post(self): # Accept POST requests
            data = request.get_json() # Get JSON data from the request body
            print(data)
            joke = data.get('joke')
            if joke is not None: # Error handling
                createJoke(joke) # Actually running the method
                return jsonify(jokes_data[-1])
            else:
                return jsonify({'error': 'Invalid data'})
    api.add_resource(_Create, '/create') # Add the method to the /create endpoint

I was able to test this using Postman:
![Postman POST Test]({{site.baseurl}}/images/postman_jokescreate.png)

## Character Songs API